In [1]:
from tensorflow import keras
import matplotlib.pyplot as plt

from typing import Optional

2022-10-12 13:50:54.700119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-12 13:50:55.604722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-12 13:50:55.604836: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-12 13:50:55.680565: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-12 13:50:56.971740: W tensorflow/stream_executor/platform/de

In [2]:
from src.data import load_data_generator
from src.regularizer import EvidenceRegularizer, EvidenceRegularizerLayer

In [3]:
dtrain, dtest = load_data_generator(name='cifar10', batch_size=1024, augment=False)

2022-10-12 13:51:14.409312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-12 13:51:14.410568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-12 13:51:14.410955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-12 13:51:14.411222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-12 13:51:14.411332: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [6]:
def cifar_10_no_regularization():
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(32,32,3)))

    model.add(keras.layers.Conv2D(32, 3, padding='same', activation='tanh'))

    model.add(keras.layers.Conv2D(32,3,padding='same', activation='tanh'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(keras.layers.Conv2D(64,3,padding='same',activation='tanh'))

    model.add(keras.layers.Conv2D(64,3,padding='same',activation='tanh'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(keras.layers.Conv2D(128,3,padding='same',activation='tanh'))

    model.add(keras.layers.Conv2D(128,3,padding='same',activation='tanh'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024,activation='tanh'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

def add_regularization(vanilla_model, regularizer = None, layers = None, **kwargs):   
    model = keras.models.Sequential()
    if layers is None:
        layers = range(1, len(vanilla_model.layers)-1)
    for i, l in enumerate(vanilla_model.layers):
        model.add(l)
        if i in layers:
            if regularizer is None:
                pass
            elif regularizer == 'l1':
                l.kernel_regularizer = keras.regularizers.l1(**kwargs)
            elif regularizer == 'l2':
                l.kernel_regularizer = keras.regularizers.l2(**kwargs)
            elif regularizer == 'dropout':
                model.add(keras.layers.Dropout(**kwargs))
            elif regularizer == 'evidence':
                l.activity_regularizer = EvidenceRegularizer(**kwargs)
            elif regularizer == 'evidence_layer':
                model.add(EvidenceRegularizerLayer(**kwargs))
            else:
                raise ValueError("Unknown regularizer: %s" % regularizer)
    return model

def create_all_models(vanilla_model_fun):
    model = vanilla_model_fun()
    dropout = add_regularization(vanilla_model=vanilla_model_fun(), regularizer='dropout', layers=[1, 4, 8, 11, 13], rate=0.2)
    evidence = add_regularization(vanilla_model=vanilla_model_fun(), regularizer='evidence', layers=[1, 4, 8, 11, 13], threshold=100, cutoff=0.0, strength=1.0)
    both = add_regularization(vanilla_model=vanilla_model_fun(), regularizer='evidence', layers=[1, 4, 8, 11, 13], threshold=100, cutoff=0.0, strength=1.0)
    both = add_regularization(vanilla_model=both, regularizer='dropout', layers=[1, 4, 8, 11, 13], rate=0.2)
    
    return {'None': model, 'Dropout': dropout, 'Evidence-based': evidence, 'Both': both}

def compile_and_fit_model(model, dtrain, dtest, compile_kwargs: Optional[dict] = None, fit_kwargs: Optional[dict] = None):
    _compile_kwargs = dict(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    if compile_kwargs is not None:
        _compile_kwargs.update(**compile_kwargs)
    model.compile(**_compile_kwargs)

    _fit_kwargs = dict(x=dtrain, validation_data=dtest, epochs=50)
    if fit_kwargs is not None:
        _fit_kwargs.update(**fit_kwargs)
    
    history = model.fit(**_fit_kwargs)

    return history
    

In [7]:
model = cifar_10_no_regularization()

In [9]:
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=1e-6, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/home/philipp/miniconda3/envs/evidence/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
history = model.fit(x=dtrain, validation_data=dtest, epochs=50)

Epoch 1/50
49/49 [==============================] - 238s 5s/step - loss: 1.9375 - accuracy: 0.3101 - val_loss: 1.6559 - val_accuracy: 0.4167
Epoch 2/50
49/49 [==============================] - 276s 6s/step - loss: 1.5342 - accuracy: 0.4555 - val_loss: 1.4457 - val_accuracy: 0.4846
Epoch 3/50
49/49 [==============================] - 285s 6s/step - loss: 1.3679 - accuracy: 0.5161 - val_loss: 1.3186 - val_accuracy: 0.5316
Epoch 4/50
49/49 [==============================] - 285s 6s/step - loss: 1.2657 - accuracy: 0.5527 - val_loss: 1.2346 - val_accuracy: 0.5633
Epoch 5/50
49/49 [==============================] - 285s 6s/step - loss: 1.1678 - accuracy: 0.5878 - val_loss: 1.1424 - val_accuracy: 0.5897
Epoch 6/50
49/49 [==============================] - 285s 6s/step - loss: 1.0817 - accuracy: 0.6214 - val_loss: 1.0668 - val_accuracy: 0.6255
Epoch 7/50
49/49 [==============================] - 284s 6s/step - loss: 1.0045 - accuracy: 0.6496 - val_loss: 1.0131 - val_accuracy: 0.6486
Epoch 8/50
49

In [12]:
models = create_all_models(cifar_10_no_regularization)

In [14]:
histories = {key: compile_and_fit_model(model=m, dtrain=dtrain, dtest=dtest) for key, m in models.items() if key != 'None'}

Epoch 1/50
49/49 [==============================] - 213s 4s/step - loss: 1.9568 - accuracy: 0.3005 - val_loss: 1.6522 - val_accuracy: 0.4186
Epoch 2/50
49/49 [==============================] - 211s 4s/step - loss: 1.5530 - accuracy: 0.4482 - val_loss: 1.4503 - val_accuracy: 0.4856
Epoch 3/50
49/49 [==============================] - 210s 4s/step - loss: 1.3906 - accuracy: 0.5066 - val_loss: 1.3414 - val_accuracy: 0.5186
Epoch 4/50
49/49 [==============================] - 209s 4s/step - loss: 1.2732 - accuracy: 0.5478 - val_loss: 1.2331 - val_accuracy: 0.5582
Epoch 5/50
49/49 [==============================] - 218s 4s/step - loss: 1.1703 - accuracy: 0.5874 - val_loss: 1.1475 - val_accuracy: 0.5895
Epoch 6/50
49/49 [==============================] - 215s 4s/step - loss: 1.0986 - accuracy: 0.6145 - val_loss: 1.0912 - val_accuracy: 0.6133
Epoch 7/50
49/49 [==============================] - 215s 4s/step - loss: 1.0134 - accuracy: 0.6464 - val_loss: 1.0249 - val_accuracy: 0.6397
Epoch 8/50
49

KeyboardInterrupt: 

In [ ]:
histories['None'] = model